# Plotting routine for the Sep 5 2022 Event - PSP

In [ ]:
test = 'test'

In [1]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocessing
import py3dcore as py3dcore

import py3dcore.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

#### setting the times for py3DCORE

In [254]:
t_launch = datetime.datetime(2022, 9, 5, 16, tzinfo=datetime.timezone.utc) # 

t_s = datetime.datetime(2022, 9, 5, 20, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2022, 9, 6, 2, 30, tzinfo=datetime.timezone.utc)

t_fit = [
        # datetime.datetime(2022, 9, 5, 18, tzinfo=datetime.timezone.utc),
       # datetime.datetime(2022, 9, 5, 19, 5, tzinfo=datetime.timezone.utc),
       # datetime.datetime(2022, 9, 5, 20, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 5, 21, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 5, 22, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 6, 0, 5, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 6, 2, 5, tzinfo=datetime.timezone.utc)
     ]


In [255]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['09/05/2022, 21:00:00', '09/05/2022, 22:00:00', '09/06/2022, 00:05:00', '09/06/2022, 02:05:00']


#### loading the pickle files

In [256]:
path = 'output/psp05092022_512_19/'

filepath = fp.loadpickle(path, number=-1)

In [257]:
filepath[:-7]

'output/psp05092022_512_19/06'

#### extracting parameters

In [258]:
res, allres, ind, meanparams = fp.get_params(filepath, give_mineps=True)

INFO:py3dcore.fluxplot:Retrieved the following parameters for the run with minimum epsilon:
INFO:py3dcore.fluxplot: --lon:  -37.63
INFO:py3dcore.fluxplot: --lat:  -43.46
INFO:py3dcore.fluxplot: --inc:  22.36
INFO:py3dcore.fluxplot: --diameter 1 AU:  0.68
INFO:py3dcore.fluxplot: --aspect ratio:  4.07
INFO:py3dcore.fluxplot: --launch radius:  7.59
INFO:py3dcore.fluxplot: --launch speed:  811.18
INFO:py3dcore.fluxplot: --t factor:  91.76
INFO:py3dcore.fluxplot: --expansion rate:  1.14
INFO:py3dcore.fluxplot: --magnetic field decay rate:  1.64
INFO:py3dcore.fluxplot: --magnetic field 1 AU:  6.72
INFO:py3dcore.fluxplot: --drag coefficient:  2.47
INFO:py3dcore.fluxplot: --background sw speed:  467.32


In [259]:
out = fp.get_ensemble_stats(filepath)

In [260]:
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,-38.444153,-31.942030,31.395012,0.514785,3.568260,8.130855,1200.858643,62.274399,7.276384,2.792043,226.401459
std,7.590301,11.512029,8.235852,0.079360,0.904775,2.149830,328.328278,50.456150,0.941131,0.767319,114.260391
min,-49.992893,-49.933163,3.077217,0.292766,1.166666,5.004879,703.260132,0.324850,5.021833,0.213721,50.363827
25%,-44.521677,-40.994778,25.433294,0.458312,2.922275,6.353864,928.757736,20.197297,6.624409,2.300780,140.846333
50%,-39.934389,-33.953352,31.345705,0.513161,3.709937,7.742208,1137.345032,50.101927,7.248016,2.886352,201.542328
75%,-33.339861,-25.110384,37.726286,0.568651,4.268652,9.533600,1447.894348,88.412600,7.881516,3.400652,290.127823
max,-8.408652,8.103820,49.581913,0.726799,4.998903,14.782323,1985.974365,246.468048,10.394197,3.994932,671.795898


In [261]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [262]:
names = ['lon: ', 'lat: ', 'inc: ', 'diameter 1 AU: ', 'aspect ratio: ', 'launch radius: ', 'launch speed: ', 
         't factor: ', 'expansion rate: ', 'magnetic field decay rate: ', 'magnetic field 1 AU: ', 'drag coefficient: ', 
         'background sw speed: ']
 
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    dfAsString = out.describe().to_string(header=True, index=True)
    f.write(dfAsString)   
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for count, name in enumerate(names):
        f.write(" --{} {:.2f}".format(name, res[count+1]))
        f.write('\n')

#### loading and plotting the fitting results

In [263]:
#t_s = datetime.datetime(2022, 9, 5, 15, tzinfo=datetime.timezone.utc) 
#t_e = datetime.datetime(2022, 9, 6, 4, tzinfo=datetime.timezone.utc)

In [264]:
fp.fullinsitu(observer='PSP', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, ref_frame='SPP_RTN', best=True, 
              ensemble=True, save_fig=True, legend=True, max_index=512, title=True, fit_points=True)

INFO:py3dcore.fluxplot:Using HelioSat to retrieve observer data


2022-09-05 20:00:00+00:00 2022-09-06 02:30:00+00:00


INFO:py3dcore.fluxplot:Using parameters for run with minimum eps.


390
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]


In [244]:
fp.scatterparams(filepath)

In [ ]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

#### Plotting the fluxrope and spacecraft positions

In [ ]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 5, 16)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist=['Earth'], t=tm1, filepath=filepath)

### make fluxrope 3d movie

In [ ]:
def make_frame(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    
    fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist =['Earth'],
              t = t, filepath = filepath, save_fig = False)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/'+frmstr+'.jpg',dpi=200)  
    return fig
    

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

### plot fluxrope 3d multiview

In [ ]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 7, 1)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d_multiview(t_launch = t_launch, filepath=filepath)

### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

